## main code

In [4]:
import requests, re
import pandas as pd
from bs4 import BeautifulSoup


# category page crawling이 안돼서 rank page에서 접속
rank_url = 'https://search.musinsa.com/ranking/best?period=now&mainCategory=001'

req = requests.get(rank_url)
document = BeautifulSoup(req.content, "html.parser")

# 메인 카테고리 수집
selector = "#goodsRankForm > div.clear > div:nth-child(16) > dl > dd > ul > li > a:nth-child(1)"
elements = document.select(selector)

title = [element.get_text() for element in elements]
ls = [re.search(('\d+'),element['href']).group() for element in elements]
code_list = {"main_title": title, "main_code" : ls}
main_code_df = pd.DataFrame(code_list)
main_code_df.head()

,main_title,main_code
0,상의,001
1,아우터,002
2,바지,003
3,원피스,020
4,스커트,022


## middle code

In [5]:
middle_code_df = pd.DataFrame(columns=['mid_title','mid_code','main_code'])


def category_crawler(sector_code='001'):
    global middle_code_df
    rank_url = 'https://search.musinsa.com/ranking/best?period=now&mainCategory={}'.format(sector_code)
    
    req = requests.get(rank_url)
    dom = BeautifulSoup(req.content, "html.parser")
    
    selector = '#goodsRankForm > div.clear > div:nth-child(17) > dl > dd > ul > li > a'
    mid_sec = dom.select(selector)
    
    sector_title = [element.get_text() for element in mid_sec]
    mid_code = [re.search(('\d+'),element['href']).group() for element in mid_sec]
    main_code = [sector_code] * len(mid_code)
    
    code_list = {"mid_title": sector_title, "mid_code" : mid_code, "main_code": main_code}
    code_list_tmp = pd.DataFrame(code_list)
    
    middle_code_df = middle_code_df.append(code_list_tmp)
    print(sector_code, middle_code_df.tail(1))

    
# 중분류 수집 : [주의] 종종 수집이 안되므로 꼭 확인 바람!!
for code in main_code_df.main_code.to_list():
    category_crawler(code)

001   mid_title mid_code main_code
8     기타 상의   001008       001
002      mid_title mid_code main_code
19  나일론/코치  재킷   002006       002
003   mid_title mid_code main_code
7     기타 바지   003006       003
020   mid_title mid_code main_code
2    맥시 원피스   020008       020
022   mid_title mid_code main_code
2      롱스커트   022003       022
004    mid_title mid_code main_code
12     클러치 백   004005       004
054   mid_title mid_code main_code
8     가방 소품   054009       054
018   mid_title mid_code main_code
3   기타 스니커즈   018004       018
007   mid_title mid_code main_code
6     기타 모자   007006       007
005   mid_title mid_code main_code
9     기타 신발   005006       005
008   mid_title mid_code main_code
1       스타킹   008002       008
026   mid_title mid_code main_code
4       홈웨어   026005       026
009   mid_title mid_code main_code
2     안경 소품   009003       009
006   mid_title mid_code main_code
4     기타 시계   006002       006
017   mid_title mid_code main_code
9    기타 스포츠   017015       017
01

In [6]:
middle_code_df.reset_index(drop=True, inplace=True)
category_df = middle_code_df.join(main_code_df.set_index('main_code'), on='main_code')
category_df

,mid_title,mid_code,main_code,main_title
0,반팔 티셔츠,001001,001,상의
1,긴팔 티셔츠,001010,001,상의
2,민소매 티셔츠,001011,001,상의
3,셔츠/블라우스,001002,001,상의
4,피케/카라 티셔츠,001003,001,상의
...,...,...,...,...
156,기타 라이프,013011,013,생활/취미/예술
157,잡지/무크지,014001,014,책/음악/티켓
158,기타 컬처,014002,014,책/음악/티켓
159,반려동물 의류,021004,021,반려동물


In [7]:
category_df.to_csv('./datas/categories.csv', index=False, encoding='UTF-8', sep=',' )

In [10]:
### dtype = 'str' 꼭 해주어야 합니다!!
df = pd.read_csv('./datas/categories.csv', dtype ='str')
df.tail()

,mid_title,mid_code,main_code,main_title
156,기타 라이프,013011,013,생활/취미/예술
157,잡지/무크지,014001,014,책/음악/티켓
158,기타 컬처,014002,014,책/음악/티켓
159,반려동물 의류,021004,021,반려동물
160,반려동물용품,021002,021,반려동물
